In [1]:
import pandas as pd
import glob

In [2]:
transcripts = pd.read_csv("data/ncte_single_utterances.csv")

In [3]:
transcripts.groupby("OBSID").size().to_dict()

{3: 250,
 4: 465,
 5: 591,
 7: 523,
 8: 468,
 9: 785,
 10: 414,
 11: 691,
 12: 394,
 13: 237,
 15: 437,
 16: 412,
 17: 564,
 18: 451,
 19: 451,
 20: 664,
 21: 166,
 22: 491,
 23: 812,
 24: 488,
 25: 496,
 26: 388,
 27: 183,
 28: 308,
 29: 289,
 30: 426,
 31: 154,
 32: 217,
 33: 162,
 34: 175,
 35: 514,
 36: 394,
 38: 498,
 39: 386,
 40: 455,
 41: 392,
 42: 319,
 44: 223,
 47: 352,
 48: 433,
 49: 175,
 50: 272,
 51: 308,
 53: 388,
 54: 473,
 56: 242,
 57: 222,
 58: 702,
 59: 584,
 60: 372,
 63: 339,
 64: 409,
 65: 268,
 66: 301,
 67: 296,
 68: 181,
 69: 377,
 70: 234,
 72: 201,
 73: 325,
 74: 284,
 75: 219,
 76: 576,
 77: 297,
 78: 423,
 79: 495,
 80: 280,
 81: 465,
 82: 441,
 83: 483,
 84: 200,
 87: 308,
 88: 403,
 89: 212,
 90: 1134,
 97: 330,
 98: 474,
 99: 329,
 100: 640,
 101: 351,
 109: 268,
 110: 555,
 111: 308,
 112: 289,
 113: 525,
 117: 261,
 118: 177,
 119: 216,
 120: 414,
 124: 140,
 127: 372,
 128: 298,
 129: 171,
 130: 57,
 131: 207,
 132: 319,
 133: 313,
 134: 481,
 136: 

In [4]:
data = pd.read_csv("data/transcript_metadata.csv")
#data["total_utterance_count"] = data["OBSID"].map(transcripts.groupby("OBSID").size().to_dict())
#data["teacher_utterance_count"] = data["OBSID"].map(transcripts[transcripts["speaker"] == "teacher"].groupby("OBSID").size().to_dict())

## Add predictions

In [5]:
def get_obsdict(data):
    data["comb_idx"] = data["comb_idx"].str.split("_")
    data["OBSID"] = [int(row[0]) for row in data["comb_idx"].tolist()]
    return data.groupby("OBSID")["pred"].sum().to_dict()

In [6]:
student_reasoning = pd.read_csv("outputs/roberta/student_reasoning_student_reasoning_candidates_preds.txt", sep='\t')

In [7]:
data["student_reasoning_pred"] = data["OBSID"].map(get_obsdict(student_reasoning))
data.loc[data["student_reasoning_pred"].isnull(), "student_reasoning_pred"] = 0

In [8]:
data["student_reasoning_pred"].isnull().sum()

0

In [9]:
# Add comb_idx because it was missing
paired_data = pd.read_csv("data/paired_utterances.csv")
for col in ["student_on_task", "teacher_on_task", "focusing_question", "high_uptake"]:
    preds = pd.read_csv("outputs/roberta/%s_paired_utterances_preds.txt" % col, sep='\t')
    preds["comb_idx"] = paired_data["comb_idx"]
    data["%s_pred" % col] = data["OBSID"].map(get_obsdict(preds))
    data.loc[data["%s_pred" % col].isnull(), "%s_pred" % col] = 0

In [10]:
data.head()

,OBSID,year,student_text,teacher_text,student_cleaned,teacher_cleaned,NCTETID,DISTRICT,CLASS_11,CLASS_12,...,SES12,SES12_count,TSP01,TSP01_count,num_chapters,student_reasoning_pred,student_on_task_pred,teacher_on_task_pred,focusing_question_pred,high_uptake_pred
0,3,1,"C. Because 16 times 3 and 16 times 6, they bot...","Okay. Math should be out. Everything else, p...",c because 16 times 3 and 16 times 6 they both ...,okay math should be out everything else please...,11024,11.0,1.0,1.0,...,0.000000,13.0,4.645161,31.0,8,10.0,21.0,22.0,7.0,14.0
1,4,1,"Yeah. Uh-huh. No. Twenty-two. Oh, okay. Yeah. ...","Red. It’s red, okay. Okay. Boys and girls, ...",yeah uh huh no twenty two oh okay yeah twenty ...,red it s red okay okay boys and girls you can ...,11054,11.0,1.0,1.0,...,0.083333,12.0,4.117647,34.0,8,10.0,30.0,24.0,4.0,15.0
2,5,1,Fractions. [Inaudible.] [Inaudible.] There’s t...,Okay. This afternoon we’re going to be doing ...,fractions inaudible inaudible there s two of t...,okay this afternoon we re going to be doing a ...,11011,11.0,1.0,1.0,...,NaN,0.0,4.227273,22.0,7,0.0,23.0,27.0,3.0,8.0
3,7,1,Good after noon Mr. C. The objective is going ...,"Okay. Okay, good afternoon boys and girls. Al...",good after noon mr c the objective is going to...,okay okay good afternoon boys and girls all ri...,11050,11.0,1.0,1.0,...,NaN,0.0,4.923077,13.0,6,15.0,83.0,80.0,21.0,22.0
4,8,1,Student C. Be – We have to really – Mine broke...,Perfect. Raise your what? I put your bag out...,student c be we have to really mine broke oh i...,perfect raise your what i put your bag outside...,11022,11.0,1.0,1.0,...,NaN,0.0,3.705882,17.0,7,10.0,55.0,55.0,7.0,21.0


In [11]:
data.to_csv("data/transcript_metadata_with_predictions.csv", index=False)

In [12]:
for c in data.columns:
    print(c)

OBSID
year
student_text
teacher_text
student_cleaned
teacher_cleaned
NCTETID
DISTRICT
CLASS_11
CLASS_12
CLASS_13
SCHOOLYEAR_SP
RATERID
OBSYEAR
OBSMONTH
OBSDAY
CHAPNUM
CLPC
CLNC
CLTS
CLRSP
CLBM
CLPRDT
CLILF
CLCU
CLAPS
CLQF
CLINSTD
CLSTENG
MQI_11
MQI_12
MQI_13
FORMAT_ACTIVE
FORMAT_BOTH
FORMAT_SMALLGRP
DIRINST
APLPROB
CWCM
WCDISS
LINK
EXPL
MMETH
MGEN
MLANG
ORICH
ORICH4
REMED
USEPROD
MATCON
OWWS
OWWS4
MAJERR
LANGIMP
LCP
OERR
OERR4
STEXPL
SMQR
ETCA
OSPMMR
OSPMMR4
STUCON
STUCOM
SMALDIS
MMSM
ORIENT
SUMM
MQI_CHECK
DIFFINST
LLC
MQI3
MKT3
MQI5
MKT5
TSTUDEA
TREMSTU
STUENG
CLMATINQ
LESSEFFIC
DENSMAT
LATASK
LESSCLEAR
TASKDEVMAT
ERRANN
WORLD
SCHOOLID
STATEVA_M
STATEVA_M09
STATEVA_M10
STATEVA_M11
STATEVA_M12
STATEVA_M13
total_utterance_count
teacher_utterance_count
STATEVA_obs_year
student_num_words
teacher_num_words
student_turn_prop
student_word_prop
SURVEYYEAR_SP
EXPERIENCE
TEACH_GRK_1
TEACH_GR2
TEACH_GR3
TEACH_GR4
TEACH_GR5
TEACH_GR6_8
MATHCOURSES
MATHCONTENT
MATHMETHODS
CERTIFICATION
MATHMAJMIN


In [13]:
len(data)

1660

## Compute cross validation results

In [19]:
k = 5
df_list = []
for col in ["student_on_task", "teacher_on_task", "high_uptake", "focusing_question", "student_reasoning"]:
    for i in range(k):
        for f in glob.glob("outputs/roberta/%s_k%d/**/eval_results.txt" %(col, i)):
            d = {
                    "col": col,
                    "accuracy": None,
                    "precision": None,
                    "recall": None,
                    "f1": None,
                }
            for score in ["accuracy", "precision", "recall", "f1"]:
                for l in open(f, "r").read().splitlines():
                    if l.startswith(score):
                        d[score] = float(l.split(" = ")[1])
            df_list.append(d)
df = pd.DataFrame(df_list)

In [20]:
print(df.groupby("col").mean().to_csv())

col,accuracy,precision,recall,f1
focusing_question,0.8564714421811914,0.47383125297508855,0.5377656850192062,0.5008947234294819
high_uptake,0.7679099941024361,0.7191022239382528,0.6744431317621904,0.6881267729520314
student_on_task,0.9020496302681125,0.9521969698950061,0.9311972489438693,0.9415573325571284
student_reasoning,0.8630419799498747,0.6442627284887887,0.6655376528999918,0.651194292864652
teacher_on_task,0.8666687837408702,0.9316829160931647,0.913878988304776,0.9226383442190219

